In [1]:
import torch
from text_generation_server.models.flash_llama import FlashLlama

model_id = "meta-llama/Llama-2-7b-hf"
model = FlashLlama(model_id=model_id, dtype=torch.bfloat16,)

You are using a model of type llama to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [2]:
!nvidia-smi

Thu Sep 28 13:30:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   28C    P0    60W / 300W |  13972MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from text_generation_server.models.flash_causal_lm import FlashCausalLMBatch
from text_generation_server.pb import generate_pb2

max_input_length = 256
max_batch_size = 32
max_prefill_tokens = max_input_length * max_batch_size - 32

warmup_requests = []
n_tokens = 0
while n_tokens < max_prefill_tokens:
    warmup_requests.append(
        generate_pb2.Request(
            id=0,
            inputs="_text" * max_input_length,
            truncate=min(max_input_length, max_prefill_tokens - n_tokens),
            parameters=generate_pb2.NextTokenChooserParameters(
                do_sample=False
            ),
            stopping_parameters=generate_pb2.StoppingCriteriaParameters(
                max_new_tokens=2
            )
        ),
    )
    
    n_tokens += max_input_length

warmup_batch = generate_pb2.Batch(id=0, requests=warmup_requests, size=len(warmup_requests))

fclm_warmup_batch = FlashCausalLMBatch.from_pb(
    pb=warmup_batch,
    tokenizer=model.tokenizer,
    dtype=model.dtype,
    device=model.device,
)

max_supported_total_tokens = model.warmup(batch=fclm_warmup_batch)

In [7]:


BATCH_SIZE = 1
MAX_NEW_TOKENS = 100

def make_clm_batch(batch_size=1, max_new_tokens=100):
    parameters = generate_pb2.NextTokenChooserParameters(
        watermark=False,
        temperature=1.0,
        repetition_penalty=1.0,
        top_k=0,
        top_p=1.0,
        typical_p=1.0,
        do_sample=False
    )

    stopping_parameters = generate_pb2.StoppingCriteriaParameters(
        max_new_tokens=MAX_NEW_TOKENS,
        ignore_eos_token=True
    )

    input_lst = [
        "In a galaxy far, far away"
    ]

    requests = [
        generate_pb2.Request(
            id=idx,
            inputs=inputs,
            truncate=max_input_length,
            parameters=parameters,    
            stopping_parameters=stopping_parameters
        )
        for idx, inputs in enumerate(input_lst * batch_size)
    ]

    return FlashCausalLMBatch.from_pb(
        pb=generate_pb2.Batch(id=0, requests=requests),
        tokenizer=model.tokenizer,
        dtype=model.dtype,
        device=model.device,
    )

texts = {
    idx: request.inputs
    for idx, request in enumerate(fclm_batch.requests)
}

for _ in range(MAX_NEW_TOKENS):
    generations, fclm_batch = model.generate_token(fclm_batch)
    for idx, gen in enumerate(generations):
        texts[idx] += gen.token_text

print(texts[0])


In a galaxy far, far away, a long time ago, a young boy named Luke Skywalker was born. Luke was a dreamer, and he dreamed of becoming a Jedi Knight. But Luke’s dreams were put on hold when his home planet of Tatooine was attacked by the evil Empire. Luke’s family was killed, and he was forced to flee to the planet of Dagobah to train with the wise Jedi Master Yoda.
Luke’s training was


In [4]:
!ls

proto  server


In [3]:
!python3 tgi-benchmarking/benchmark.py

python3: can't open file '/usr/src/tgi-benchmarking/benchmark.py': [Errno 2] No such file or directory
